In [1]:
import os
import math
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt


In [2]:
# single household contsumption data extract function
def extract_house(data_frame, lclid):
    df = data_frame.set_index("tstp")
    df.index = df.index.astype("datetime64")
    
    df["energy(kWh/hh)"] = df["energy(kWh/hh)"].astype("float64")
    df = df[df["LCLid"] == "{}".format(lclid) ]
    return df

# weather extraction function
def extract_weather(data_frame):
    df = data_frame.set_index("time")
    df.index = df.index.astype("datetime64")
    return df

## Elia grid load data

In [50]:
elia = pd.read_csv("ELIA_LOAD_2020.csv",header = 1)

In [51]:
elia.head()

,dd,mm,yyyy,0:15,0:30,0:45,1:00,1:15,1:30,1:45,...,22:45,23:00,23:15,23:30,23:45,24:00,"2:15""","2:30""","2:45""","3:00"""
0,1,1,2020,8209192,8155125,8084900,8016324,7985205,7886804,7799053,...,8555191,8551015,8692071,8616506,8500310,8351431,0,0,0,0
1,2,1,2020,8248012,8158287,8042245,7942309,7905350,7768966,7691843,...,8743621,8624613,8775540,8678593,8510042,8373122,0,0,0,0
2,3,1,2020,8228368,8103054,7922370,7773588,7719466,7503258,7426806,...,9083141,9081538,9206615,9084543,8956135,8845601,0,0,0,0
3,4,1,2020,8771361,8680820,8548456,8437740,8291005,8146864,8035014,...,8995693,8907230,9047341,8866118,8779609,8730643,0,0,0,0
4,5,1,2020,8605184,8544523,8350719,8254368,8157975,8032916,7889415,...,9021058,8873602,8998122,8971952,8842632,8704938,0,0,0,0


In [52]:
elia.drop(columns=['2:15"','2:30"','2:45"','3:00"'],inplace=True)


In [53]:
elia.replace('NOT VALID', np.NaN, inplace=True)

In [54]:
elia.head()

,dd,mm,yyyy,0:15,0:30,0:45,1:00,1:15,1:30,1:45,...,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45,24:00
0,1,1,2020,8209192,8155125,8084900,8016324,7985205,7886804,7799053,...,8204405,8161784,8357563,8517970,8555191,8551015,8692071,8616506,8500310,8351431
1,2,1,2020,8248012,8158287,8042245,7942309,7905350,7768966,7691843,...,8476694,8392585,8531725,8643723,8743621,8624613,8775540,8678593,8510042,8373122
2,3,1,2020,8228368,8103054,7922370,7773588,7719466,7503258,7426806,...,8804174,8805315,8999869,9163460,9083141,9081538,9206615,9084543,8956135,8845601
3,4,1,2020,8771361,8680820,8548456,8437740,8291005,8146864,8035014,...,8766813,8646281,8735102,8921705,8995693,8907230,9047341,8866118,8779609,8730643
4,5,1,2020,8605184,8544523,8350719,8254368,8157975,8032916,7889415,...,8903514,8843971,8909111,9059377,9021058,8873602,8998122,8971952,8842632,8704938


In [55]:
date_list = []
for i in range(elia.shape[0]):
    date_list.append("{}-{}-{}".format(elia['yyyy'][i],elia['mm'][i],elia['dd'][i]))

In [56]:
elia["YYYY-MM-DD"] = date_list

In [58]:
elia.drop(columns = ['dd','mm','yyyy'], inplace = True)

In [61]:
elia.set_index(elia["YYYY-MM-DD"], inplace= True)

In [63]:
elia.drop(columns = ['YYYY-MM-DD'], inplace=True)
elia.head()

,0:15,0:30,0:45,1:00,1:15,1:30,1:45,2:00,2:15,2:30,...,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45,24:00
YYYY-MM-DD,,,,,,,,,,,,,,,,,,,,,
2020-1-1,8209192,8155125,8084900,8016324,7985205,7886804,7799053,7702752,7643799,7564089,...,8204405,8161784,8357563,8517970,8555191,8551015,8692071,8616506,8500310,8351431
2020-1-2,8248012,8158287,8042245,7942309,7905350,7768966,7691843,7584363,7536363,7486732,...,8476694,8392585,8531725,8643723,8743621,8624613,8775540,8678593,8510042,8373122
2020-1-3,8228368,8103054,7922370,7773588,7719466,7503258,7426806,7336997,7276770,7227533,...,8804174,8805315,8999869,9163460,9083141,9081538,9206615,9084543,8956135,8845601
2020-1-4,8771361,8680820,8548456,8437740,8291005,8146864,8035014,7888652,7732757,7589089,...,8766813,8646281,8735102,8921705,8995693,8907230,9047341,8866118,8779609,8730643
2020-1-5,8605184,8544523,8350719,8254368,8157975,8032916,7889415,7754012,7732688,7678821,...,8903514,8843971,8909111,9059377,9021058,8873602,8998122,8971952,8842632,8704938


In [125]:
## Expanse the data set
index_list = []
energy_list = []
for date in elia.index:
    for time in elia.columns:
        #special case handling
        if time == "24:00":
            index_list.append("{} {}".format(date, "00:00"))
        else:
            index_list.append("{} {}".format(date, time))
        energy_list.append(elia.loc[date, time])
elia_qh = pd.DataFrame(list(zip(index_list,energy_list)),  columns=['time','grid_load'])
elia_qh.head(100)


,time,grid_load
0,2020-1-1 0:15,8209192
1,2020-1-1 0:30,8155125
2,2020-1-1 0:45,8084900
3,2020-1-1 1:00,8016324
4,2020-1-1 1:15,7985205
...,...,...
95,2020-1-1 00:00,8351431
96,2020-1-2 0:15,8248012
97,2020-1-2 0:30,8158287
98,2020-1-2 0:45,8042245


In [126]:
elia_qh['time'] = elia_qh['time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M"))

/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


In [127]:
elia_qh["grid_load"] = elia_qh["grid_load"].astype("float64")
elia_qh['time'] = elia_qh['time'].astype("datetime64")


0:15     object
0:30      int64
0:45      int64
1:00      int64
1:15      int64
          ...  
23:00    object
23:15    object
23:30    object
23:45    object
24:00    object
Length: 96, dtype: object

In [128]:
elia_qh.dtypes

time         datetime64[ns]
grid_load           float64
dtype: object

In [135]:
elia_qh.set_index(elia_qh['time'], inplace = True)
elia_qh.drop(columns = ['time'], inplace = True)

In [137]:
elia_qh.to_csv("elia_qh.csv", index = True)

### Electricity consumption processing

In [16]:
elec = pd.read_csv("../smart-meters-in-london/halfhourly_dataset/block_0.csv")
for num in range(1,111):    
    elec = elec.append(pd.read_csv("../smart-meters-in-london/halfhourly_dataset/block_"+str(num)+".csv"))
        

/Users/fuchucheng/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
elec.replace('Null', np.NaN, inplace=True)
energy = elec.iloc[:,:]
energy["energy"] = energy["energy(kWh/hh)"].astype("float64")
energy = energy.drop(["energy(kWh/hh)"],axis = 1)

**Missing values**

In [18]:
def missing_count(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: data[df_cols[i]].isnull().sum()})
    
    return dict_x


missing = missing_count(energy)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 5544), ('LCLid', 0), ('tstp', 0)]

### Single household

In [20]:
energy_MAC1 = energy[energy["LCLid"] == "MAC000010"]

In [21]:
missing = missing_count(energy_MAC1)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]

Count of missing data


[('energy', 1), ('LCLid', 0), ('tstp', 0)]

In [22]:
energy_MAC1["energy"] = energy_MAC1["energy"].astype("float64")
energy_MAC1["tstp"] = energy_MAC1["tstp"].astype("datetime64")
energy_MAC1 = energy_MAC1.groupby('tstp').agg({'energy': lambda x: x.mean(skipna=False)})
energy_MAC1['tstp'] = energy_MAC1.index
energy_MAC1.drop(['tstp'], axis = 1, inplace=True)


/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### To prevent data leakage we split trian and test first

In [30]:
train_s = energy_MAC1.iloc[:20000, :]
test_s = energy_MAC1.iloc[20000:,:]

**then deal with missing value half-hr**

In [31]:
train_s.fillna(train_s.median(), inplace = True)
test_s.fillna(test_s.median(), inplace = True)

**resampling to 1hr**

In [32]:
train_s_h = train_s.resample('1H').sum()
test_s_h = test_s.resample('1H').sum()

In [36]:
##Save to csv
train_s.to_csv("train_s_hh.csv", index = True)
test_s.to_csv("train_s_hh.csv", index = True)
train_s_h.to_csv("train_s_h.csv", index = True)
test_s_h.to_csv("train_s_h.csv", index = True)

### Aggregated   household

**Group by LCLid and aggregate over per hour**


In [29]:
## deal with missing value by interpolating mean of whole column
# energy["energy"].fillna(energy["energy"].median(), inplace = True)
## group by time stamps and aggregate through mean
energy["energy"] = energy["energy"].astype("float64")
energy["tstp"] = energy["tstp"].astype("datetime64")
energy = energy.groupby('tstp').agg({'energy': lambda x: x.mean(skipna=False)})
energy['tstp'] = energy.index
energy.drop(['tstp'], axis = 1, inplace=True)

### To prevent data leakage we split trian and test first

In [6]:
train_e = energy.iloc[:28750, :]
test_e = energy.iloc[28750:,:]


**deal with missing value sampling rate: 30 mins**

In [7]:
train_e.fillna(train_e.median(), inplace = True)
test_e.fillna(test_e.median(), inplace = True)


//miniconda3/envs/tpot/lib/python3.6/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
## check if there is still missing value
missing = missing_count(test_e)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 0)]

**then deal with missing value and resampling to 1hr**

In [31]:
train_e.fillna(train_e.median(), inplace = True)
test_e.fillna(test_e.median(), inplace = True)
train_e = train_e.resample('1H').sum()
test_e = test_e.resample('1H').sum()

//miniconda3/envs/TPOT/lib/python3.5/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
## check if there is still missing value
missing = missing_count(train_e)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 673)]

### Weather data processing


In [9]:
weather = pd.read_csv('../smart-meters-in-london/weather_hourly_darksky.csv')

In [10]:
## set index as time stamp
weather = weather.set_index("time")
weather.index = weather.index.astype("datetime64[ns]")

In [11]:
## apparently the sequence in between is not correct
weather = weather.sort_index()

**Missing values**

In [12]:
def count_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().sum()*100,2)})
    
    return dict_x

missing = count_missing(weather)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:]

Percent of missing data


[('pressure', 1300),
 ('visibility', 0),
 ('windBearing', 0),
 ('temperature', 0),
 ('dewPoint', 0),
 ('apparentTemperature', 0),
 ('windSpeed', 0),
 ('precipType', 0),
 ('icon', 0),
 ('humidity', 0),
 ('summary', 0)]

In [13]:
## drop columns wont be used later
weather.drop(['precipType','icon','summary'], axis = 1, inplace=True)

In [14]:
##merge trian and test seperately
train =  weather.merge(train_e, how ='right', left_index = True, right_index = True)
test = weather.merge(test_e.iloc[1:,:], how = 'right', left_index = True, right_index = True)

In [16]:
train.to_csv("hh_train_nan.csv", index = True)
test.to_csv("hh_test_nan.csv", index = True)

In [12]:
train = pd.read_csv("hh_train_nan.csv", index_col='tstp')
test = pd.read_csv("hh_test_nan.csv", index_col='tstp')

In [39]:
## interpolate nan
for col in train.columns:
    train[col] = train[col].interpolate(method='linear')
for col in test.columns:
    test[col] = test[col].interpolate(method = "linear")

In [43]:
## data set without catagorical features
train.to_csv("train_nocat_hh.csv", index = True)
test.to_csv("test_nocat_hh.csv", index = True)

**One hot encoding**

In [73]:
## Examine unique categories in categorical features
p = list(weather["precipType"].unique()) 
i = list(weather["icon"].unique()) 
u = list(weather["summary"].unique())
# Sort them for one hot encoding labels
p.sort()
i.sort()
u.sort()

In [74]:
from sklearn.preprocessing import OneHotEncoder
## One hot encoding these features above
onehot = OneHotEncoder(dtype=np.int, sparse=True)
nominals = pd.DataFrame(onehot.fit_transform(weather[['precipType', 'icon','summary']]).toarray(),columns=p+i+u)
nominals.index = weather.index

In [12]:
#Concate nominals with origin weather dataframe
weather = weather.set_index("time")
weather.index = weather.index.astype("datetime64")
#weather.drop(["precipType","icon","summary"],axis = 1, inplace = True)
weather =  weather.merge(nominals, left_index = True, right_index = True)


KeyError: "None of ['time'] are in the columns"

### Merge weather and energy data

In [14]:
### Merge weather and energy data together
weather_energy = weather.merge(energy, left_index = True, right_index = True)
weather_energy.index.names = ["time"]

In [15]:
##Split data into training set and testing set and save to csv file format
train_we = weather_energy.iloc[:14000,:]
test_we = weather_energy.iloc[14000:,:]
train_we.to_csv("train_auto.csv", index = True)
test_we.to_csv("test_auto.csv", index = True)

In [177]:
##Split data into training set and testing set and save to csv file format
train_we = weather_energy.iloc[:14000,:]
test_we = weather_energy.iloc[14000:,:]
train_we.to_csv("train.csv", index = True)
test_we.to_csv("test.csv", index = True)

In [176]:
weather_energy

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,humidity,rain,snow,...,Clear,Foggy,Mostly Cloudy,Overcast,Partly Cloudy,Windy,Windy and Mostly Cloudy,Windy and Overcast,Windy and Partly Cloudy,energy
time,,,,,,,,,,,,,,,,,,,,,
2011-12-11 00:00:00,12.50,210,2.83,1.17,1015.67,1.11,1.78,0.89,1,0,...,0,0,0,0,1,0,0,0,0,0.498395
2011-12-11 01:00:00,12.65,204,2.48,0.81,1014.96,0.31,2.11,0.89,1,0,...,1,0,0,0,0,0,0,0,0,0.445535
2011-12-11 02:00:00,13.02,214,2.70,1.29,1014.42,0.11,2.57,0.90,1,0,...,1,0,0,0,0,0,0,0,0,0.349090
2011-12-11 03:00:00,13.05,211,3.47,1.41,1013.78,0.66,3.00,0.86,1,0,...,0,0,0,0,1,0,0,0,0,0.317786
2011-12-11 04:00:00,12.97,204,3.74,1.53,1012.94,1.29,2.64,0.85,1,0,...,0,0,0,0,1,0,0,0,0,0.281382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-02-15 19:00:00,12.68,247,7.39,3.06,997.07,3.91,6.08,0.74,1,0,...,0,0,0,0,1,0,0,0,0,0.744099
2014-02-15 20:00:00,13.78,249,6.56,2.76,998.15,3.03,5.61,0.77,1,0,...,1,0,0,0,0,0,0,0,0,0.685877
2014-02-15 21:00:00,14.31,248,6.47,2.71,999.28,3.06,5.25,0.77,1,0,...,1,0,0,0,0,0,0,0,0,0.634185
